In [45]:
from sklearn.metrics import pairwise_distances
from sklearn.metrics import euclidean_distances
from math import ceil, log2
import numpy as np
import time

In [8]:
from scipy import spatial

def dist(x, y):
    return spatial.distance.jaccard(x, y)

x = [0,0,0,0]
y = [1,0,1,0]

dist(x,y)

1.0

In [11]:
data = [
    [1,0,0,0],
    [1,0,1,0],
    
    [0,0,0,0],
    [0,1,1,1],
    
    [0,1,0,1],
]

cs = [
    [0,0,0,0],
    [1,0,1,0],
]

for d in data:
    for c in cs:
        print(d, ' - ', c, ': ', dist(d,c))
    print('--')

[1, 0, 0, 0]  -  [0, 0, 0, 0] :  1.0
[1, 0, 0, 0]  -  [1, 0, 1, 0] :  0.5
--
[1, 0, 1, 0]  -  [0, 0, 0, 0] :  1.0
[1, 0, 1, 0]  -  [1, 0, 1, 0] :  0.0
--
[0, 0, 0, 0]  -  [0, 0, 0, 0] :  0
[0, 0, 0, 0]  -  [1, 0, 1, 0] :  1.0
--
[0, 1, 1, 1]  -  [0, 0, 0, 0] :  1.0
[0, 1, 1, 1]  -  [1, 0, 1, 0] :  0.75
--
[0, 1, 0, 1]  -  [0, 0, 0, 0] :  1.0
[0, 1, 0, 1]  -  [1, 0, 1, 0] :  1.0
--


In [16]:
new = [data[2], data[4]]
for i in range(2):
    for j in range(2):
        print(i,j,dist(new[i],new[j]))

0 0 0
0 1 1.0
1 0 1.0
1 1 0.0


In [ ]:
https://people.revoledu.com/kardi/tutorial/kMean/Online-K-Means-Clustering.html

In [51]:
# calculate I
def I(a,b):
    if (a * b) == 0:
        return 0
    a = a * 1.0
    b = b * 1.0
    n = a + b
    return -(a * log2(a/n) + b * log2(b/n)) / n

# calculate split I
def SplitI(arr):
    n = sum(arr) 
    result = 0
    for x in arr:
        if x == 0:
            result += 0
        result += x * log2(x/n)
    return -result / n

In [59]:
(4*I(2,2)+1*I(1,0)+3*I(0,3))/8

0.5

In [53]:
SplitI([2,1,2])

1.5219280948873624

In [60]:
def betacv(data, labels, metric='cosine'):
    distances = pairwise_distances(data, metric=metric)
    n = labels.shape[0]
    A = np.array([intra_cluster_distance(distances[i], labels, i)
                  for i in range(n)])
    B = np.array([inter_cluster_distance(distances[i], labels, i)
                  for i in range(n)])
    a = np.sum(A) / 2.0
    b = np.sum(B) / 2.0
    labels_unq = np.unique(labels)
    members = np.array([member_count(labels, i) for i in labels_unq])
    N_in = np.array([i*(i-1) for i in members])
    n_in = np.sum(N_in) / 2.0
    N_out = np.array([i*(n-i) for i in members])
    n_out = np.sum(N_out) / 2.0
    betacv = (a/n_in)/(b/n_out)
    print('-----------')
    print('beta:', betacv)
    print('intra:', a)
    print('inter:', b)
    print('n_in :', n_in)
    print('n_out:', n_out)
    return betacv

def intra_cluster_distance(distances_row, labels, i):
    mask = labels == labels[i]
    mask[i] = False
    if not np.any(mask):
        return 0
    print(f'w_in for [{i+1}]: ', mask)
    a = np.sum(distances_row[mask])
    print(' => w = ', a)
    return a

def inter_cluster_distance(distances_row, labels, i):
    mask = labels != labels[i]
    b = np.sum(distances_row[mask])
    print(f'w_out for [{i+1}]: ', mask)
    return b

def member_count(labels, i):
    mask = labels == i
    return len(labels[mask])

In [61]:
betacv(
    data=np.array([
        [0,0,1,1,1],
        [1,1,0,0,0],
        [0,1,1,0,0],
        [1,1,0,1,0],
        [1,0,1,0,1],
        [0,1,0,1,0]
    ]),
    labels=np.array([0,1,1,1,0,1])
)

w_in for [1]:  [False False False False  True False]
 => w =  0.33333333333333315
w_in for [2]:  [False False  True  True False  True]
 => w =  1.1835034190722742
w_in for [3]:  [False  True False  True False  True]
 => w =  1.5917517095361373
w_in for [4]:  [False  True  True False False  True]
 => w =  0.9587585476806849
w_in for [5]:  [ True False False False False False]
 => w =  0.33333333333333315
w_in for [6]:  [False  True  True  True False False]
 => w =  1.1835034190722742
w_out for [1]:  [False  True  True  True False  True]
w_out for [2]:  [ True False False False  True False]
w_out for [3]:  [ True False False False  True False]
w_out for [4]:  [ True False False False  True False]
w_out for [5]:  [False  True  True  True False  True]
w_out for [6]:  [ True False False False  True False]
-----------
beta: 0.5597845135096582
intra: 2.7920918810140183
inter: 5.7003401714778805
n_in : 7.0
n_out: 8.0


0.5597845135096582

In [16]:
!python apriori.py -f data.csv -s 0.6 -c 0.8

Accepted item:  frozenset({'D'}) 0.6666666666666666
Accepted item:  frozenset({'A'}) 0.6666666666666666
Accepted item:  frozenset({'W'}) 0.8333333333333334
Accepted item:  frozenset({'T'}) 0.6666666666666666
Accepted item:  frozenset({'C'}) 1.0
Rejected item:  frozenset({'W', 'D'}) 0.5
Accepted item:  frozenset({'C', 'A'}) 0.6666666666666666
Rejected item:  frozenset({'W', 'T'}) 0.5
Accepted item:  frozenset({'W', 'A'}) 0.6666666666666666
Accepted item:  frozenset({'C', 'T'}) 0.6666666666666666
Rejected item:  frozenset({'T', 'D'}) 0.3333333333333333
Accepted item:  frozenset({'C', 'D'}) 0.6666666666666666
Rejected item:  frozenset({'T', 'A'}) 0.5
Accepted item:  frozenset({'W', 'C'}) 0.8333333333333334
Rejected item:  frozenset({'A', 'D'}) 0.3333333333333333
Rejected item:  frozenset({'C', 'A', 'D'}) 0.3333333333333333
Rejected item:  frozenset({'C', 'T', 'A'}) 0.5
Rejected item:  frozenset({'C', 'T', 'D'}) 0.3333333333333333
Accepted item:  frozenset({'W', 'C', 'A'}) 0.66666666666666

In [18]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

dataset = [
    ['A', 'C', 'T', 'W'],
    ['C', 'D', 'W'],
    ['C', 'D', 'T'],
    ['A', 'C', 'T', 'W'],
    ['A', 'C', 'D', 'W'],
    ['A', 'C', 'D', 'T', 'W'],
]
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,A,C,D,T,W
0,True,True,False,True,True
1,False,True,True,False,True
2,False,True,True,True,False
3,True,True,False,True,True
4,True,True,True,False,True
5,True,True,True,True,True


In [20]:
from mlxtend.frequent_patterns import fpgrowth

fpgrowth(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,1.000000,(C)
1,0.833333,(W)
2,0.666667,(T)
3,0.666667,(A)
4,0.666667,(D)
5,0.833333,"(C, W)"
6,0.666667,"(C, T)"
7,0.666667,"(A, W)"
8,0.666667,"(C, A)"
9,0.666667,"(C, A, W)"


In [23]:
from mlxtend.frequent_patterns import apriori
apriori(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.666667,(A)
1,1.000000,(C)
2,0.666667,(D)
3,0.666667,(T)
4,0.833333,(W)
5,0.666667,"(C, A)"
6,0.666667,"(A, W)"
7,0.666667,"(C, D)"
8,0.666667,"(C, T)"
9,0.833333,"(C, W)"
